# KfServing Logger demo

We create a message dumper KNaive service to print out CloudEvents it receives:

In [1]:
!pygmentize message-dumper.yaml

apiVersion: serving.knative.dev/v1
kind: Service
metadata:
  name: message-dumper
spec:
  template:
    spec:
      containers:
      - image: gcr.io/knative-releases/knative.dev/eventing-contrib/cmd/event_display


In [53]:
!kubectl apply -f message-dumper.yaml -n default

service.serving.knative.dev/message-dumper created


Create a Torchserve model with associated logger to push events to the message logger URL

In [26]:
!pygmentize torchserve-logger.yaml

apiVersion: "serving.kubeflow.org/v1beta1"
kind: "InferenceService"
metadata:
  name: "torchserve-custom"
spec:
  predictor:
    minReplicas: 1      
    logger:
      url: http://message-dumper.default/
      port: 8091
      component-port: 8083
      mode: all
    containers:
      - image: jagadeeshj/ts_logger:latest
        name: torchserve-container


In [28]:
!kubectl apply -f torchserve-logger.yaml -n kfserving-test

error: error validating "torchserve-logger.yaml": error validating data: [ValidationError(InferenceService.spec.predictor.logger): unknown field "component-port" in org.kubeflow.serving.v1beta1.InferenceService.spec.predictor.logger, ValidationError(InferenceService.spec.predictor.logger): unknown field "port" in org.kubeflow.serving.v1beta1.InferenceService.spec.predictor.logger]; if you choose to ignore these errors, turn validation off with --validate=false


In [18]:
CLUSTER_IPS=!(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].hostname}')
CLUSTER_IP=CLUSTER_IPS[0]
print(CLUSTER_IP)

a881f5a8c676a41edbccdb0a394a80d6-2069247558.us-west-2.elb.amazonaws.com


In [17]:
SERVICE_HOSTNAMES=!(kubectl get inferenceservice torchserve-custom -n kfserving-test -o jsonpath='{.status.url}' | cut -d "/" -f 3)
SERVICE_HOSTNAME=SERVICE_HOSTNAMES[0]
print(SERVICE_HOSTNAME)

torchserve-custom.kfserving-test.a881f5a8c676a41edbccdb0a394a80d6-2069247558.us-west-2.elb.amazonaws.com


In [6]:
!wget https://raw.githubusercontent.com/pytorch/serve/master/examples/image_classifier/mnist/test_data/1.png

--2020-11-25 14:07:23--  https://raw.githubusercontent.com/pytorch/serve/master/examples/image_classifier/mnist/test_data/1.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.252.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.252.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167 [image/png]
Saving to: ‘1.png’

1.png               100%[===================>]     167  --.-KB/s    in 0s      

2020-11-25 14:07:24 (5.86 MB/s) - ‘1.png’ saved [167/167]



In [19]:
!echo $SERVICE_HOSTNAME

torchserve-custom.kfserving-test.a881f5a8c676a41edbccdb0a394a80d6-2069247558.us-west-2.elb.amazonaws.com


In [20]:
!curl -v -H "Host: $SERVICE_HOSTNAME" http://$CLUSTER_IP/predictions/mnist -T 1.png

*   Trying 52.37.128.207...
* Connected to a881f5a8c676a41edbccdb0a394a80d6-2069247558.us-west-2.elb.amazonaws.com (52.37.128.207) port 80 (#0)
> PUT /predictions/mnist HTTP/1.1
> Host: torchserve-custom.kfserving-test.a881f5a8c676a41edbccdb0a394a80d6-2069247558.us-west-2.elb.amazonaws.com
> User-Agent: curl/7.47.0
> Accept: */*
> Content-Length: 167
> Expect: 100-continue
> 
< HTTP/1.1 100 Continue
* We are completely uploaded and fine
< HTTP/1.1 200 OK
< content-length: 1
< content-type: text/plain; charset=utf-8
< date: Wed, 25 Nov 2020 08:47:23 GMT
< x-envoy-upstream-service-time: 5097
< server: istio-envoy
< 
* Connection #0 to host a881f5a8c676a41edbccdb0a394a80d6-2069247558.us-west-2.elb.amazonaws.com left intact
1

In [22]:
!kubectl logs $(kubectl get pod -l serving.knative.dev/configuration=message-dumper -n default -o jsonpath='{.items[0].metadata.name}') -c user-container 

☁️  cloudevents.Event
Validation: valid
Context Attributes,
  specversion: 1.0
  type: org.kubeflow.serving.inference.request
  source: http://localhost:8081/
  id: d89f208a-ce47-443b-b2c2-de8f6917d6be
  time: 2020-11-25T08:47:18.251273118Z
  datacontenttype: application/json
Extensions,
  endpoint: 
  inferenceservicename: 
  namespace: kfserving-test
  traceparent: 00-8e32641b959177c0d17ec23338e1310c-5f6d4019418c5c29-00
Data,


IHDR          Wf�H   nIDATx�c`�`��$�r�o�n�)���$��߻��e��q�'y��G?<�rpJ����[2�N��p����?�%=����-���Hj���_�l����%U�I
����)I-  �J4ԩ��    IEND�B`�
☁️  cloudevents.Event
Validation: valid
Context Attributes,
  specversion: 1.0
  type: org.kubeflow.serving.inference.response
  source: http://localhost:8081/
  id: d89f208a-ce47-443b-b2c2-de8f6917d6be
  time: 2020-11-25T08:47:23.347753905Z
  datacontenttype: application/json
Extensions,
  endpoint: 
  inferenceservicename: 
  namespace: kfserving-test
  traceparent: 00-2ee61b93151cd79fffc7bba52c4b56e8-4dd5

In [13]:
!kubectl delete -f torchserve-logger.yaml

inferenceservice.serving.kubeflow.org "sklearn-iris" deleted


In [14]:
!kubectl delete -f message-dumper.yaml

service.serving.knative.dev "message-dumper" deleted
